# WiDS

## Data Loading

__Importing the libraries__

In [1]:
from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler,
    FunctionTransformer,
)

from sklearn.impute import SimpleImputer
# from sklearn.impute import IterativeImputer

from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer


from sklearn.model_selection import (
    cross_val_score,
    cross_validate,
    train_test_split,
    RandomizedSearchCV
)

from sklearn.linear_model import Ridge

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from catboost import CatBoostRegressor
from lightgbm.sklearn import LGBMRegressor
from xgboost import XGBRegressor

from sklearn.ensemble import StackingClassifier

from sklearn.metrics import make_scorer
# from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve

In [2]:
import numpy as np
import pandas as pd
from torchvision import transforms, datasets, utils
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split, Dataset
from torch import Tensor
from torch.nn import Linear
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import MSELoss
import torch.utils.data as data
from torch import nn

In [3]:
train_path = "data/train.csv"
test_path = "data/test.csv"

In [4]:
class Dataprep(Dataset):

    def __init__(self, train_path, test_path, size=0.1, is_train=True):
        df = pd.read_csv(train_path)
        test_df = pd.read_csv(test_path)

        TARGET_COLUMN = "site_eui"

        self.is_train = is_train

        train_df, val_df = train_test_split(
            df,
            test_size=0.1,
            random_state=123
        )

        self.X_train, self.y_train = train_df.drop(columns=[TARGET_COLUMN]), train_df[TARGET_COLUMN]
        self.X_val, self.y_val = val_df.drop(columns=[TARGET_COLUMN]), val_df[TARGET_COLUMN]
        self.X_test = test_df

    def engineer_features(self):
        # Find Standard Deviation of min, max and avg temp among months
        min_temps = [
            "january_min_temp",
            "february_min_temp",
            "march_min_temp",
            "april_min_temp",
            "may_min_temp",
            "june_min_temp",
            "july_min_temp",
            "august_min_temp",
            "september_min_temp",
            "october_min_temp",
            "november_min_temp",
            "december_min_temp"
        ]

        max_temps = [
            "january_max_temp",
            "february_max_temp",
            "march_max_temp",
            "april_max_temp",
            "may_max_temp",
            "june_max_temp",
            "july_max_temp",
            "august_max_temp",
            "september_max_temp",
            "october_max_temp",
            "november_max_temp",
            "december_max_temp"
        ]

        avg_temps = [
            "january_avg_temp",
            "february_avg_temp",
            "march_avg_temp",
            "april_avg_temp",
            "may_avg_temp",
            "june_avg_temp",
            "july_avg_temp",
            "august_avg_temp",
            "september_avg_temp",
            "october_avg_temp",
            "november_avg_temp",
            "december_avg_temp",
        ]

        self.numeric_features = [
            "floor_area",
            "year_built",
            "energy_star_rating",
            "ELEVATION",
            "cooling_degree_days",
            "heating_degree_days",
            "precipitation_inches",
            "snowfall_inches",
            "snowdepth_inches",
            "avg_temp",
            "days_below_30F",
            "days_below_20F",
            "days_below_10F",
            "days_below_0F",
            "days_above_80F",
            "days_above_90F",
            "days_above_100F",
            "days_above_110F",
            "max_wind_speed",
            "days_with_fog",
            "building_age",
            "min_temp_std",
            "max_temp_std",
            "avg_temp_std",
            "0-10",
            "10-20",
            "20-30",
            "30-80",
            "80-90",
            "90-100",
            "100-110"
        ] + min_temps + max_temps + avg_temps

        self.categorical_features = [
            "Year_Factor",
            "State_Factor",
            "building_class",
            "facility_type",
            "direction_max_wind_speed",
            "direction_peak_wind_speed"
        ]

        self.drop_columns = [
            "id"
        ]
        
        for X in [self.X_train, self.X_val, self.X_test]:
            X["building_age"] = 2022 - X["year_built"]

        for X in [self.X_train, self.X_val, self.X_test]:
            X["min_temp_std"] = X[min_temps].T.std()
            X["max_temp_std"] = X[max_temps].T.std()
            X["avg_temp_std"] = X[avg_temps].T.std()

        days_above_below = [
            "days_below_30F",
            "days_below_20F",
            "days_below_10F",
            "days_below_0F",
            "days_above_80F",
            "days_above_90F",
            "days_above_100F",
            "days_above_110F",
        ]

        for X in [self.X_train, self.X_val, self.X_test]:
            X["0-10"] = X["days_below_10F"] - X["days_below_0F"]
            X["10-20"] = X["days_below_20F"] - X["days_below_10F"]
            X["20-30"] = X["days_below_30F"] - X["days_below_20F"]
            X["80-90"] = X["days_above_80F"] - X["days_above_90F"]
            X["90-100"] = X["days_above_90F"] - X["days_above_100F"]
            X["100-110"] = X["days_above_100F"] - X["days_above_110F"]
            X["30-80"] = (366 - X[days_above_below].sum(axis=1)).clip(lower=0)

    def preprocess(self):
        pipe_numeric_feats = make_pipeline(
           SimpleImputer(strategy="mean"),
           StandardScaler()
        )
        pipe_cat_feats = make_pipeline(
            SimpleImputer(strategy="most_frequent"),
            OneHotEncoder(handle_unknown="ignore", sparse=False)
        )
        self.column_transformer = make_column_transformer(
            (pipe_numeric_feats, self.numeric_features),
            (pipe_cat_feats, self.categorical_features)
        )
        
        self.X_train_raw = self.X_train
        self.X_val_raw = self.X_val
        self.X_test_raw = self.X_test
        self.X_train = self.column_transformer.fit_transform(self.X_train)
        self.X_val = self.column_transformer.transform(self.X_val)
        self.X_test = self.column_transformer.transform(self.X_test)

        self.X_train_tensor = torch.tensor(self.X_train, dtype=torch.float32)
        self.y_train_tensor = torch.tensor(self.y_train.values, dtype=torch.float32)
        self.X_val_tensor = torch.tensor(self.X_val, dtype=torch.float32)
        self.y_val_tensor = torch.tensor(self.y_val.values, dtype=torch.float32)
        self.X_test_tensor = torch.tensor(self.X_test, dtype=torch.float32)

In [5]:
feature_set = Dataprep(train_path, test_path)
feature_set.engineer_features()
feature_set.preprocess()

In [6]:
def mse(predictions, targets):
    return np.mean((predictions - targets) ** 2)
        
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

def mape(true, pred): 
    return 100.0 * np.mean(np.abs((pred - true) / true))

def r2score_torch(predictions, target):
    target_mean = torch.mean(target)
    ss_tot = torch.sum((target - target_mean) ** 2)
    ss_res = torch.sum((target - predictions) ** 2)
    r2 = 1 - ss_res / ss_tot
    return r2

def mse_torch(predictions, targets):
    return torch.mean((predictions - targets) ** 2)
        
def rmse_torch(predictions, targets):
    return torch.sqrt(((predictions - targets) ** 2).mean())

## Tree Models

In [ ]:
for dt in [
    feature_set.X_train,
    feature_set.X_val,
    feature_set.y_train,
    feature_set.y_val
]:
    print(dt.shape)

In [ ]:
def cross_val_scores(model, X_train, y_train, X_val, y_val, return_train_score=False):

    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)

    score_dict = {
        "r2_val": model.score(X_val, y_val),
        "mse_val": mse(y_val, y_val_pred),
        "rmse_val": rmse(y_val, y_val_pred),
        "mape_val": mape(y_val, y_val_pred)
    }

    if return_train_score:
        y_train_pred = model.predict(X_train)

        score_dict["r2_train"] = model.score(X_train, y_train)
        score_dict["mse_train"] = mse(y_train, y_train_pred)
        score_dict["rmse_train"] = rmse(y_train, y_train_pred)
        score_dict["mape_train"] = mape(y_train, y_train_pred)

    scores_result = pd.Series(score_dict)

    return model, scores_result

In [ ]:
pipe_lr = make_pipeline(feature_set.column_transformer, Ridge(max_iter=10000))
pipe_rf = make_pipeline(feature_set.column_transformer, RandomForestRegressor())
pipe_xgb = make_pipeline(feature_set.column_transformer, XGBRegressor(verbosity=0))
pipe_lgbm = make_pipeline(feature_set.column_transformer, LGBMRegressor())
pipe_catboost = make_pipeline(feature_set.column_transformer, CatBoostRegressor(verbose=False))

In [ ]:
models = {
    "Ridge": pipe_lr,
    "Random Forest": pipe_rf,
    "XGB": pipe_xgb,
    "LGBM": pipe_lgbm,
    "Cat Boost": pipe_catboost
}

In [ ]:
results = {}

for name, model in models.items():
    print(f"Start {name}!")
    _, results[name] = cross_val_scores(
        model,
        feature_set.X_train_raw,
        feature_set.y_train,
        feature_set.X_val_raw,
        feature_set.y_val,
        return_train_score=True
    )

    print(f"Done {name}!")


In [ ]:
pd.DataFrame(results)

In [ ]:
scoring_metrics = {
    "neg RMSE": "neg_root_mean_squared_error",
    "r2": "r2"
}

In [ ]:
# Hyperparam Tune Random Forest 

params = {
    'randomforestregressor__n_estimators': [10, 100, 1000],
    'randomforestregressor__max_depth': [5, 10, 12],
    'randomforestregressor__max_features': ['auto', 'sqrt']
}

In [ ]:
# random_search.fit(X_train, y_train)

In [ ]:
# pd.DataFrame(random_search.cv_results)

In [ ]:
# random_search.best_params_

In [ ]:
# random_search.best_score_

In [ ]:
# HyperparamTune XGBoost
params_xgb = {
    'xgbregressor__n_estimators': [10, 100, 1000],
    'xgbregressor__max_depth': [3, 5, 7, 12],
    'xgbregressor__eta': [0.01, 0.03, 0.01, 0.3]
}

In [ ]:
 random_search = RandomizedSearchCV(
    pipe_xgb,
    params_xgb,
    n_jobs=-1,
    return_train_score=True,
    scoring=scoring_metrics,
    refit="r2"
)

In [ ]:
# results_xgb = random_search.fit(X_train, y_train)

In [ ]:
pd.DataFrame(results_xgb)

# FCNN

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [8]:
def linear_block(input_size, output_size):
    return nn.Sequential(
        nn.Linear(input_size, output_size),
        nn.ReLU(),
        nn.Dropout(0.2)
    )


class EnergyRegressor(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.layers = nn.Sequential(
            linear_block(input_size, 2 * input_size),
            linear_block(2 * input_size, 5 * input_size),
            linear_block(5 * input_size, 10 * input_size),
            linear_block(10 * input_size, 7 * input_size),
            linear_block(7 * input_size, 5 * input_size),
            linear_block(5 * input_size, 3 * input_size),
            linear_block(3 * input_size, input_size),
            nn.Linear(input_size, 1000),
            nn.Linear(1000, 700),
            nn.Linear(700, 400), 
            nn.Linear(400, 256),
            nn.Linear(256, 128),
            nn.Linear(128, 64),
            nn.Linear(64, 1)
        )

    def forward(self, X):
        X = X.to(device)
        X = self.layers(X)
        return X

In [9]:
trainloader = DataLoader(TensorDataset(feature_set.X_train_tensor, feature_set.y_train_tensor), batch_size=32, shuffle=True)
validloader = DataLoader(TensorDataset(feature_set.X_val_tensor, feature_set.y_val_tensor), batch_size=32, shuffle=True)

In [10]:
model = EnergyRegressor(feature_set.X_train_tensor.shape[1])
model.to(device)

In [11]:
def trainer(model, criterion, optimizer, trainloader, validloader, epochs):
    train_mse = 0
    train_rmse = 0
    train_r2 = 0
    val_mse = 0
    val_rmse = 0
    val_r2 = 0
    
    for epoch in range(epochs):
        train_batch_mse = []
        train_batch_rmse = []
        train_batch_r2 = []
        val_batch_mse = []
        val_batch_rmse = []
        val_batch_r2 = []

        model.train(True)

        for X, y in trainloader:
            X = X.to(device)
            y = y.to(device)
            y_hat = model(X).flatten()
            
            optimizer.zero_grad()
            loss = criterion(y_hat, y)
            loss = loss.to(device)
            loss.backward()
            optimizer.step()
            mse_train = mse_torch(y_hat, y)
            rmse_train = rmse_torch(y_hat, y)
            r2_train = r2score_torch(y_hat, y)
            train_batch_mse.append(mse_train)
            train_batch_rmse.append(rmse_train)
            train_batch_r2.append(r2_train)
        
        train_mse = torch.sum(torch.Tensor(train_batch_mse)) / len(trainloader)
        train_rmse = torch.sum(torch.Tensor(train_batch_rmse)) / len(trainloader)
        train_r2 = torch.sum(torch.Tensor(train_batch_r2)) / len(trainloader)

        model.eval()

        with torch.no_grad():
            for X_valid, y_valid in validloader:
                X_valid = X_valid.to(device)
                y_valid = y_valid.to(device).flatten()
                y_hat_val = model(X_valid)
                mse_val = mse_torch(y_hat_val, y_valid)
                rmse_val = rmse_torch(y_hat_val, y_valid)
                r2_val = r2score_torch(y_hat_val, y_valid)
                val_batch_mse.append(mse_val)
                val_batch_rmse.append(rmse_val)
                val_batch_r2.append(r2_val)
            val_mse = torch.sum(torch.Tensor(val_batch_mse)) / len(validloader)
            val_rmse = torch.sum(torch.Tensor(val_batch_rmse)) / len(validloader)
            val_r2 = torch.sum(torch.Tensor(val_batch_r2)) / len(validloader) 

        print(f"Epoch {epoch + 1}:\tTrain MSE: {round(train_mse, 4)}. Train RMSE: {round(train_rmse, 4)}, Train R2: {round(train_r2, 4)}.")
        print(f"\t\tVal MSE: {round(val_mse, 4)}, Val RMSE: {round(val_rmse, 4)}, Val R2: {round(val_r2, 4)}.")
        print("-" * 50)
    return model

In [12]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = criterion.to(device)
trained_model = trainer(model, criterion, optimizer, trainloader, validloader, epochs=100)

In [13]:
def predict(model, X):
    return model(X.type(torch.float32))

In [14]:
predictions = predict(trained_model, feature_set.X_test_tensor)
predictions

In [ ]:
feature_set.X_test_raw.shape

In [ ]:
feature_set.X_test.shape

In [ ]:
results_dict = {"id": feature_set.X_test_raw["id"],
               "site_eui": predictions.cpu().detach().numpy().flatten()}
pd.DataFrame(results_dict).set_index("id").to_csv("submission.csv")